"使用任何神经网络框架，对CIFAR-10进行分类
http://www.cs.toronto.edu/~kriz/cifar.html
训练集 50000，测试集 10000
图像大小 32*32 彩色
10个分类：ariplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck"

In [1]:
import torch
from torch import nn, optim
import torchvision
import torch.nn.functional as F

In [70]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.conv = nn.Sequential(
            # 32*28*28
            nn.Conv2d(3, 32, 3, 1), # in_channels, out_channels, kernel_size, stride, padding
            nn.MaxPool2d(2, 2), # kernel_size, stride
            nn.ReLU(),            
            # 减小卷积窗口，使用填充为2来使得输入与输出的高和宽一致，且增加输出通道数
            # 64*14*14
            nn.Conv2d(32, 64, 3, 1, 1),
            nn.MaxPool2d(2, 2),
            nn.ReLU(),
        # 连续3个卷积层，且使用更小的卷积窗口。除了最后的卷积层外，进一步 增加了输出通道数。
        # 前两个卷积层后不使用池化层来减小输入的⾼和宽 
            # 128*7*7
            nn.Conv2d(64, 128, 3, 1, 1), 
            # 256*7*7
            nn.Conv2d(128, 256, 3, 1, 1), 
            # 256*7*7
            nn.Conv2d(256, 256, 3, 1, 1), 
            # 256*3*3
            nn.MaxPool2d(2, 2),
            nn.ReLU()
        )
        # 这里全连接层的输出个数 ALexNet中的数倍。使用dropout来缓解过拟合。
        self.fc = nn.Sequential(
        # 输出层，10分类
            nn.Linear(256*3*3, 1024),
            nn.Linear(1024, 512),
            nn.Linear(512, 10),
        )
    def forward(self, img):
        feature = self.conv(img)
        output = self.fc(feature.view(img.shape[0], -1))
        return output

In [60]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
# 超参数定义
EPOCH = 10               # 训练epoch次数
BATCH_SIZE = 64         # 批训练的数量
LR = 0.001              # 学习率

In [11]:
train_data = datasets.CIFAR10(root='CIFAR/',train=True,transform=transforms.ToTensor(),download=True)
test_data = datasets.CIFAR10(root='CIFAR/',train=False,transform=transforms.ToTensor(),download=True)

Files already downloaded and verified
Files already downloaded and verified


In [79]:
train_data.data.shape

(50000, 32, 32, 3)

In [82]:
len(train_data.targets)

50000

In [71]:
# 使用DataLoader进行分批
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE, shuffle=True)

In [72]:
# AlexNet Model
model = AlexNet()
#损失函数:这里用交叉熵
criterion = nn.CrossEntropyLoss()
#优化器 这里用SGD
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [73]:
#device : GPU or CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

AlexNet(
  (conv): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): ReLU()
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): ReLU()
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): ReLU()
  )
  (fc): Sequential(
    (0): Linear(in_features=2304, out_features=1024, bias=True)
    (1): Linear(in_features=1024, out_features=512, bias=True)
    (2): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [74]:
# 训练
for epoch in range(EPOCH):
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        # 前向传播
        outputs = model(inputs)
        # 计算损失函数
        loss = criterion(outputs, labels)
        # 清空上一轮梯度
        optimizer.zero_grad()
        # 反向传播
        loss.backward()
        # 参数更新
        optimizer.step()
    print('epoch{} loss:{:.4f}'.format(epoch+1, loss.item()))

epoch1 loss:1.4124
epoch2 loss:1.4200
epoch3 loss:1.0555
epoch4 loss:1.4278
epoch5 loss:0.4376
epoch6 loss:0.8953
epoch7 loss:0.9961
epoch8 loss:1.0440
epoch9 loss:0.7370
epoch10 loss:0.5514


In [75]:
#保存训练模型
torch.save(model, 'CIFAR.pt')

/opt/conda/lib/python3.7/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type AlexNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [76]:
model = torch.load('CIFAR.pt')

# 测试
model.eval()
correct = 0
total = 0

In [77]:
for data in test_loader:
    images, labels = data
    images, labels = images.to(device), labels.to(device)
    # 前向传播
    out = model(images)
    _, predicted = torch.max(out.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

In [78]:
#输出识别准确率
print('准确率:{:.4f}%'.format(100 * correct / total)) 

10000测试图像 准确率:67.5600%
